In [ ]:
!pip install -q -U PyPDF2  accelerate   peft

In [ ]:
import json
import csv
import os
import pandas as pd
import random
import string
import PyPDF2
import io
import sys
import traceback



def extract_text_from_pdf(filepath, length_of_chunk):
    try:
        # Open the PDF file in read-binary mode
        with open(filepath, 'rb') as file:
            # Initialize a PDF file reader object
            reader = PyPDF2.PdfReader(file)

            # Initialize an empty string to hold the extracted text
            text = ''

            # Loop through each page in the PDF
            for page in reader.pages:
                # Extract the text from the page and add it to the text string
                text += page.extract_text()

        # Split the text into chunks of the specified length
        chunks = [text[i:i + length_of_chunk] for i in range(0, len(text), length_of_chunk)]

        # Return the chunks
        return chunks

    except FileNotFoundError:
        print(f"The file {filepath} does not exist.")
        return None
    except:
        print("An unexpected error occurred.")
        return None

def get_module_structure(directory_path):
  """ Generates a list of all files and their module structure within a given directory.

  Args:
    directory_path (str): Path to the directory you want to explore.

  Returns:
    A dictionary representing the module structure, with keys as module names and values as lists of file names.
  """

  module_structure = {}

  for root, dirs, files in os.walk(directory_path):
     # Normalize the root path to ensure consistent separators
    normalized_root = os.path.normpath(root).replace('\\', '/')
        # Get the module name from the directory path
    module_name = os.path.basename(normalized_root)
    # Add the module name to the dictionary if it doesn't exist
    if module_name not in module_structure:
      module_structure[module_name] = []

    # Add the file names to the list for the module
    module_structure[module_name].extend([os.path.join(normalized_root, file).replace('\\', '/') for file in files])

  return module_structure

def get_module_csv(directory_path):
    """ Generates a list of all .csv files and their module structure within a given directory.

    Args:
      directory_path (str): Path to the directory you want to explore.

    Returns:
      A dictionary representing the module structure, with keys as module names and values as lists of .csv file names.
    """

    module_csv = {}

    for root, dirs, files in os.walk(directory_path):
        # Normalize the root path to ensure consistent separators
        normalized_root = os.path.normpath(root).replace('\\', '/')
        # Get the module name from the directory path
        module_name = os.path.basename(normalized_root)

        # Add the module name to the dictionary if it doesn't exist
        if module_name not in module_csv:
            module_csv[module_name] = []

        # Add only .csv file names to the list for the module
        csv_files = [file for file in files if file.lower().endswith('.csv')]
        module_csv[module_name].extend([os.path.join(normalized_root, file).replace('\\', '/') for file in csv_files])

    return module_csv


def get_module_json(directory_path):
    """ Generates a list of all .json files and their module structure within a given directory.

    Args:
      directory_path (str): Path to the directory you want to explore.

    Returns:
      A dictionary representing the module structure, with keys as module names and values as lists of .json file names.
    """
    module_json = {}

    for root, dirs, files in os.walk(directory_path):
        # Normalize the root path to ensure consistent separators
        normalized_root = os.path.normpath(root).replace('\\', '/')

        # Get the module name from the directory path
        module_name = os.path.basename(normalized_root)

        # Add the module name to the dictionary if it doesn't exist
        if module_name not in module_json:
            module_json[module_name] = []

        # Add only .json file names to the list for the module
        json_files = [file for file in files if file.lower().endswith('.json')]
        module_json[module_name].extend([os.path.join(normalized_root, file).replace('\\', '/') for file in json_files])

    return module_json

def normalize_json_data(data):
    """
    Normalizes JSON data to a list of dictionaries if possible.
    """
    # If it's a dictionary, wrap it in a list
    if isinstance(data, dict):
        return [data]
    # If it's a list of lists or primitives, convert it to a list of dictionaries
    elif isinstance(data, list) and all(not isinstance(item, dict) for item in data):
        # Assume each item in the list can be a row in the CSV
        return [{str(index): value for index, value in enumerate(item)} if isinstance(item, list) else {'value': item} for item in data]
    # If it's already a list of dictionaries, no need to modify
    elif isinstance(data, list) and all(isinstance(item, dict) for item in data):
        return data
    else:
        raise ValueError('JSON data structure is not supported.')
def json_to_csv(json_file_path, csv_file_path):
    """
    Converts a JSON file into a CSV file.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        normalized_data = normalize_json_data(data)
        headers = set().union(*(d.keys() for d in normalized_data))
        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(normalized_data)
        print(f"Converted JSON to CSV: {csv_file_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")
        traceback.print_exc()

# def json_to_csv(json_file_path, csv_file_path):
#     """
#     Converts a JSON file into a CSV file.

#     Args:
#       json_file_path (str): Path to the input JSON file.
#       csv_file_path (str): Path to the output CSV file.
#     """
#     try:
#         # Check if JSON file exists
#         if not os.path.isfile(json_file_path):
#             raise FileNotFoundError(f"JSON file does not exist: {json_file_path}")

#         # Read JSON data
#         with open(json_file_path, 'r', encoding='utf-8') as json_file:
#             data = json.load(json_file)

#         # Normalize the JSON data to a list of dictionaries
#         normalized_data = normalize_json_data(data)

#         # Aggregate headers from all data entries
#         headers = {key for item in normalized_data for key in item.keys()}

#         # Write CSV data
#         with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
#             csv_writer = csv.DictWriter(csv_file, fieldnames=headers)
#             csv_writer.writeheader()  # Write the headers to the CSV file
#             csv_writer.writerows(normalized_data)  # Write data rows

#         print(f"Successfully converted {json_file_path} to {csv_file_path}.")

#     except json.JSONDecodeError:
#         print(f"Error: Invalid JSON data in file {json_file_path}.")
#     except FileNotFoundError as e:
#         print(str(e))
#     except ValueError as e:
#         print(str(e))
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")



def print_random_row(dataframe):
    """
    Prints a random row from a Pandas DataFrame.

    Args:
      dataframe (pd.DataFrame): A Pandas DataFrame object.

    Returns:
      str: A formatted string of the random row.
    """
    if dataframe.empty:
        return "The DataFrame is empty."

    random_index = random.randint(0, len(dataframe) - 1)
    random_row = dataframe.iloc[random_index]

    task = "\n".join(f"{column}: {value}" for column, value in random_row.items())
    prompt = f"Following the instructions:\n{task}"
    return prompt
def chunkify(text, length_of_chunk):
    """
    Splits text into chunks of a given length.

    Args:
        text (str): The text to be split.
        length_of_chunk (int): The maximum length of each chunk.

    Returns:
        list of str: A list of text chunks.
    """
    return [text[i:i+length_of_chunk] for i in range(0, len(text), length_of_chunk)]
def get_all_rows(dataframe, length_of_chunk=2048):
    """
    Returns all rows from a Pandas DataFrame in specified chunk lengths.

    Args:
        dataframe (pd.DataFrame): A Pandas DataFrame object.
        length_of_chunk (int): The maximum length of each chunk.

    Returns:
        List[str]: A list of formatted strings of all the rows, chunked by length.
    """
    if dataframe.empty:
        return ["The DataFrame is empty."]

    all_chunks = []
    for index, row in dataframe.iterrows():
        # Format the row into a string
        row_str = "\n".join(f"{column}: {value}" for column, value in row.items())
        prompt = f"Row {index}:\n{row_str}\n"
        # Split the formatted row string into chunks
        row_chunks = chunkify(prompt, length_of_chunk)
        all_chunks.extend(row_chunks)

    return all_chunks

def process_json_file(json_file_path, length_of_chunk=2048):
    # Convert the JSON file to CSV format
    csv_file_path = json_file_path.replace('.json', '.csv')
    json_to_csv(json_file_path, csv_file_path)

    # Load the CSV into a pandas DataFrame
    dataframe = pd.read_csv(csv_file_path)

    # Get all rows from the DataFrame, formatted and chunked
    return get_all_rows(dataframe, length_of_chunk)

def preprocess_text(text):
    """
    Preprocesses the text.

    Args:
      text (str): The text to preprocess.

    Returns:
      str: The preprocessed text.
    """
    # Example: simple preprocessing that removes punctuation and converts to lowercase.
    return text.translate(str.maketrans('', '', string.punctuation)).lower()

def process_files_text(file_path, length_of_chunk=2048):
    """
    Processes a file and returns a list of tuples containing the starting byte and the preprocessed chunk of text.

    Args:
      file_path (str): The path to the file to process.
      length_of_chunk (int): The size of each chunk to be read and processed.

    Returns:
      List[Tuple[int, str]]: A list of tuples containing the starting byte and the preprocessed chunk of text.
    """
    # Check if file exists and is a file
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file at path {file_path} does not exist.")

    processed_chunks = []

    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            start_byte = file.tell()  # Get the current position in the file
            chunk = file.read(length_of_chunk)
            if not chunk:
                break  # End of file reached
            preprocessed_chunk = preprocess_text(chunk)
            processed_chunks.append((start_byte, preprocessed_chunk))
            if len(chunk) < length_of_chunk:
                break  # Last chunk, smaller than max size, processed

    return processed_chunks
# def process_files(file_path, length_of_chunk=200):
#     try:
#         file_extension = get_file_extension(file_path)
#         if not file_extension:
#             # Optionally handle files with no extension differently here
#             raise ValueError(f'Error processing file {file_path}: File has no extension')
#         if file_extension == '.pdf':
#             return extract_text_from_pdf(file_path, length_of_chunk)
#         elif file_extension in ['.json', '.csv']:
#             return process_json_file(file_path, length_of_chunk)
#         elif file_extension in ['.txt', '.md', '.py', '.ipynb']:
#             return process_files_text(file_path, length_of_chunk)
#         else:
#             raise NotImplementedError(f'File type {file_extension} is not supported.')
#     except (ValueError, NotImplementedError) as e:
#         # Log error and continue with the next file
#         print(f"Skipping file {file_path} due to error: {e}")
#         return None

def get_file_extension(file_path):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower()

def process_files(file_path, length_of_chunk=2048):
    """
    Processes a file and returns a list of processed chunks of text.
    """
    try:
        file_extension = get_file_extension(file_path)
        if not file_extension:
            raise ValueError(f'File {file_path} has no extension.')

        if file_extension == '.pdf':
            return extract_text_from_pdf(file_path, length_of_chunk)
        elif file_extension == '.json':
            return process_json_file(file_path, length_of_chunk)
        elif file_extension == '.csv':
            dataframe = pd.read_csv(file_path)
            return get_all_rows(dataframe, length_of_chunk)
        elif file_extension in ['.txt', '.md', '.py']:
            return process_files_text(file_path, length_of_chunk)
        else:
            raise NotImplementedError(f'File type {file_extension} is not supported.')

    except FileNotFoundError:
        print(f'File not found: {file_path}')
    except ValueError as ve:
        print(f'ValueError: {ve}')
    except NotImplementedError as nie:
        print(f'NotImplementedError: {nie}')
    except Exception as e:
        print(f'An unexpected error occurred while processing {file_path}: {e}')
        traceback.print_exc()

# directory_path = "/content/drive/MyDrive"
# try:
#     module_structure = get_module_structure(directory_path)
#     for module, files in module_structure.items():
#         print(f'Module: {module}')
#         for file in files:
#             try:
#                 result = process_files(file, length_of_chunk=1000)
#                 # Only print the result if it's not None
#                 if result is not None:
#                     print("\n", result[:1][0])
#             except Exception as e:
#                 # Catch any other unexpected exceptions and log them
#                 print(f"An error occurred while processing {file}: {e}")
#         print("\n")
# except Exception as e:
#     print(f"An error occurred while getting the module structure: {e}")


In [ ]:
!pip install -q -U transformers

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Assuming process_files returns a processed text or list of texts

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, file_paths, length_of_chunk):
        self.tokenizer = tokenizer
        self.texts = []
        self.length_of_chunk = length_of_chunk
        self.file_paths = file_paths
        for file_path in file_paths:
            self.texts.extend(self._load_and_process_file(file_path))

    def _load_and_process_file(self, file_path):
        processed_data = process_files(file_path, self.length_of_chunk)
        if processed_data is None:
            return []
        return processed_data

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.tokenizer(self.texts[idx], return_tensors='pt', truncation=True, padding='max_length', max_length=512)

# Initialize tokenizer and model
model_name = 'gpt2'  # for example, use the appropriate model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Get file paths from the module structure
directory_path = "/content/drive/MyDrive/Applied-Deep-Learning/02 - Natural Language Processing"
file_paths = []
try:
    module_structure = get_module_structure(directory_path)
    for module, files in module_structure.items():
        file_paths.extend([os.path.join(module, f) for f in files])
except Exception as e:
    print(f"An error occurred while getting the module structure: {e}")

# Create the Custom Dataset
dataset = CustomTextDataset(tokenizer, file_paths, length_of_chunk=1000)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)  # Adjust batch size according to your GPU capacity

# # Define Trainer and Training Arguments
# training_args = TrainingArguments(
#     output_dir="./model_output",  # Set to your desired output directory
#     per_device_train_batch_size=1,  # Adjust based on your GPU memory and model size
#     logging_dir='./logs',
#     logging_steps=10,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
#     data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'][0] for f in data]),
#                                 'attention_mask': torch.stack([f['attention_mask'][0] for f in data]),
#                                 'labels': torch.stack([f['input_ids'][0] for f in data])},
# )

# # Start the training
# trainer.train()

# # Save the model
# trainer.save_model(output_dir='./model_output')

In [ ]:
!pip install -q -U transformers --upgrade

In [ ]:
# Define Trainer and Training Arguments
training_args = TrainingArguments(
    output_dir="./model_output",  # The output directory for model checkpoints and predictions
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    do_train=True,  # Whether to run training
    do_eval=True,  # Whether to run evaluation on the dev dataset
    do_predict=True,  # Whether to run predictions on the test dataset
    evaluation_strategy="steps",  # The evaluation strategy to adopt during training: steps, epoch, or no
    prediction_loss_only=True,  # Whether to only return the loss when predicting
    per_device_train_batch_size=4,  # Batch size per device during training, adjust based on GPU memory
    per_device_eval_batch_size=4,  # Batch size per device for evaluation, adjust based on GPU memory
    gradient_accumulation_steps=1,  # Number of updates steps to accumulate before performing a backward pass
    eval_accumulation_steps=None,  # Number of predictions steps to accumulate before moving the tensors to the CPU
    eval_delay=None,  # The delay for evaluation in seconds (to throttle the frequency of eval loops)
    learning_rate=5e-5,  # The initial learning rate for Adam
    weight_decay=0,  # Weight decay for Adam
    adam_beta1=0.9,  # Beta1 for Adam
    adam_beta2=0.999,  # Beta2 for Adam
    adam_epsilon=1e-8,  # Epsilon for Adam
    max_grad_norm=1.0,  # Max gradient norm
    num_train_epochs=3,  # Total number of training epochs to perform
    max_steps=-1,  # If > 0: set total number of training steps to perform. Override num_train_epochs
    lr_scheduler_type="linear",  # The scheduler type to use: linear, cosine, etc
    warmup_ratio=0.0,  # Linear warmup over warmup_ratio fraction of total steps
    warmup_steps=0,  # Linear warmup over warmup_steps
    log_level="info",  # Logger verbosity
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every X updates steps
    save_strategy="steps",  # The checkpoint save strategy to adopt during training: steps, epoch, no
    save_steps=500,  # Save checkpoint every X updates steps
    save_total_limit=None,  # Limit the total amount of checkpoints. Deletes the older checkpoints
    seed=42,  # Random seed for initialization
    disable_tqdm=False,  # Whether to disable the progress bars
    load_best_model_at_end=True,  # Whether to load the best model found at each evaluation
    metric_for_best_model="loss",  # The metric to use to compare two different models
    greater_is_better=False,  # Whether the `metric_for_best_model` should be maximized or not
    dataloader_num_workers=0,  # Number of subprocesses to use for data loading
    label_smoothing_factor=0.0,  # The label smoothing epsilon to use (0 means no label smoothing)
    # Add any additional arguments you need here
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'][0] for f in data]),
                                'attention_mask': torch.stack([f['attention_mask'][0] for f in data]),
                                'labels': torch.stack([f['input_ids'][0] for f in data])},
    # Add any additional Trainer configuration here
)

# Start the training
trainer.train()

# Save the model
trainer.save_model(output_dir='./model_output')

In [ ]:
!

In [ ]:
pip install -q -U transformers==4.35.2

In [ ]:
pip install accelerate -U

In [ ]:
!git clone https://github.com/myshell-ai/OpenVoice.git

In [ ]:
!cd OpenVoice/
!ls

In [ ]:
!pip install -q -U -r  requirements.txt

In [ ]:
!python api.py

In [ ]:
import os
import torch
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter

In [ ]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device = 'cuda:0'
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name = 'gpt2'

In [ ]:
# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
pip install accelerate

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # device_map=device_map,
    low_cpu_mem_usage=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])